Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares. 
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqt4ba44yx)

De [aca](http://www.astronexus.com/hyg) saco un csv de estrellas. Lo cargo como DataFrame para leerlo mas facil. Preproceso el DF y luego obtengo el RDD.

In [ ]:
from faker import Faker
from pyspark.sql.functions import udf
from pyspark.sql.types import *
faker = Faker()

df = spark.read.csv("textos/hygfull.csv.gz",header=True).select(["StarId","Mag","Spectrum"])
udf_timestamp = udf(lambda x: faker.unix_time(),IntegerType())
udf_spectrum = udf(lambda x: x.strip()[0] if len(x.strip())>0 else "",StringType())
df = df.withColumn("timestamp",udf_timestamp("StarId"))
df = df.withColumn("Spectrum",udf_spectrum("Spectrum"))
df = df.withColumn("Mag",df["Mag"].cast(DoubleType()))
df = df.withColumn("StarId",df["StarId"].cast(IntegerType()))
df = df.filter(df.Spectrum != "")
df.columns

In [ ]:
rdd = df.rdd.map(tuple)

Necesitamos agrupar por tipo espectral (tercer columna) y promedio de magnitud (segunda columna). Nos piden que el promedio de magnitud este redondeado a un decimal. Creamos la clave para agrupar y como valor el id. Agrupamos por clave:

In [ ]:
rdd = rdd.map(lambda x: ((round(x[1],1),x[2]),x[0])).groupByKey()

Ahora tenemos algo del tipo <clave, iterable de StarIds>. Queremos convertir ese iterable a una lista. Una opcion seria hacer `.map(lambda x: (x[0], list(x[1])))`, pero es mucho mas prolijo hacerlo usando [.mapValues](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=mapvalues#pyspark.RDD.mapValues):

In [ ]:
rdd = rdd.mapValues(list).map(lambda x: x[1])

Por lo que dice el enunciado, no importa la clave, solo la lista de similares. Entonces en la celda anterior nos quedamos con solo la lista de similares de cada registro.

In [ ]:
rdd.collect()